# Convert TIFF Time Series to OME-Zarr (Fiji Compatible)

This notebook reads a sequence of 3D TIFF images (e.g., a time series), stacks them as a 3D array with axes `TCZYX`, and saves the result as a Fiji-compatible OME-Zarr dataset using `ome-zarr-py`.


In [1]:
# Mount Google Drive (for Colab users)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install required packages (run only once)
!pip install tifffile dask ome-zarr zarr imagecodecs --quiet

## 1. Import Required Libraries

We will use `tifffile` for reading TIFFs, `dask` for lazy evaluation, and `ome-zarr` to write the output in the OME-NGFF format.


In [3]:
import os
#import tifffile
import zarr
#import dask.array as da
#import matplotlib.pyplot as plt
#import numpy as np

#from ome_zarr.io import parse_url
#from ome_zarr.writer import write_image
from ome_zarr.format import CurrentFormat
#from dask import delayed


import tifffile
import dask.array as da
import numpy as np
from ome_zarr.writer import write_image
from ome_zarr.io import parse_url
import matplotlib.pyplot as plt

## 2. Define Input and Output Paths

- `tiff_folder`: Folder containing the TIFF image sequence (each image is a time point).
- `output_zarr`: Path for the resulting OME-Zarr dataset.


In [4]:
# Define paths (edit these)
#tiff_folder = "/content/drive/MyDrive/2025/Estudiantes/2025 Rodrigo Escobedo/250520/sampleOPCdata"
#zarr_path = "/content/drive/MyDrive/2025/Estudiantes/2025 Rodrigo Escobedo/250520/sampleOPCdata.ome.zarr"

tiff_folder = "/content/drive/MyDrive/2025/Estudiantes/2025 Rodrigo Escobedo/250520/Exp3_MIB2"
zarr_path = "/content/drive/MyDrive/2025/Estudiantes/2025 Rodrigo Escobedo/250520/Exp3_MIB2.ome.zarr"



## 4. Collect and Sort the TIFF Files


In [5]:
tiff_files = sorted([
    os.path.join(tiff_folder, f)
    for f in os.listdir(tiff_folder)
    if f.lower().endswith(('.tif', '.tiff'))
])
print(f"Found {len(tiff_files)} TIFF files.")

Found 9001 TIFF files.


## 5. Read and write the Dataset as OME-Zarr to Google Drive

In [6]:

# Cargar en Dask (aún con tifffile, pero evitando RAM completa)
lazy_arrays = [da.from_array(tifffile.memmap(f), chunks="auto") for f in tiff_files]
stack = da.stack(lazy_arrays)  # shape: (T, Y, X)

# Expandir a (T, C, Z, Y, X) si es necesario
stack = stack[:, None, None, :, :]  # agrega ejes C y Z

# Escribir como OME-Zarr

store = zarr.DirectoryStore(zarr_path)
root_group = zarr.group(store=store, overwrite=True)
write_image(
    image=stack,
    group=root_group,
    axes="tczyx",
    fmt=CurrentFormat(),
    overwrite=True
)
print(f"OME-Zarr saved to: {zarr_path}")

OME-Zarr saved to: /content/drive/MyDrive/2025/Estudiantes/2025 Rodrigo Escobedo/250520/Exp3_MIB2.ome.zarr


## 5. Load Dask stack from the OME-Zarr container

In [ ]:
# Open the Zarr group directly
root = zarr.open(zarr_path, mode='r')

# List available groups (resolutions)
print("Zarr groups:", list(root.group_keys()))

# Access the multiscale image at highest resolution (level 0)
main_image = root['0']

# Print metadata
print("Shape:", main_image.shape)
print("Dtype:", main_image.dtype)
print("Min:", np.min(main_image[:]))
print("Max:", np.max(main_image[:]))

Zarr groups: []
Shape: (9001, 1, 1, 480, 640)
Dtype: uint16


## 6. Open and visualize first and mean projection

In [ ]:
# Assume grayscale: stack shape is (T, 1, 1, Y, X)
# Extract first image
first_image = main_image[10, 0, 0, :, :]  # shape (Y, X)

# Compute mean projection over time
mean_projection = np.mean(main_image[0:49, 0, 0, :, :], axis=0)

# Plotting
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

ax[0].imshow(first_image, cmap='gray')
ax[0].set_title("First Timepoint")
ax[0].axis('off')

ax[1].imshow(mean_projection, cmap='gray')
ax[1].set_title("Mean Projection Over Time")
ax[1].axis('off')

plt.tight_layout()
plt.show()